In [1]:
%pip install accelerate bitsandbytes safetensors

Note: you may need to restart the kernel to use updated packages.


In [2]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModel

config = AutoConfig.from_pretrained("google/flan-ul2")

with init_empty_weights():
  model = AutoModel.from_config(config)

device_map = infer_auto_device_map(model)

/home/miboj/miniconda3/envs/summarisation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
device_map["lm_head.weight"] = 'cpu'
device_map

{'shared': 0,
 'decoder.embed_tokens': 0,
 'encoder.embed_tokens': 0,
 'encoder.block.0': 0,
 'encoder.block.1': 0,
 'encoder.block.2': 0,
 'encoder.block.3': 0,
 'encoder.block.4': 0,
 'encoder.block.5': 0,
 'encoder.block.6': 0,
 'encoder.block.7.layer.0': 0,
 'encoder.block.7.layer.1.DenseReluDense.wi_0': 0,
 'encoder.block.7.layer.1.DenseReluDense.wi_1': 'cpu',
 'encoder.block.7.layer.1.DenseReluDense.wo': 'cpu',
 'encoder.block.7.layer.1.DenseReluDense.dropout': 'cpu',
 'encoder.block.7.layer.1.DenseReluDense.act': 'cpu',
 'encoder.block.7.layer.1.layer_norm': 'cpu',
 'encoder.block.7.layer.1.dropout': 'cpu',
 'encoder.block.8': 'cpu',
 'encoder.block.9': 'cpu',
 'encoder.block.10': 'cpu',
 'encoder.block.11': 'cpu',
 'encoder.block.12': 'cpu',
 'encoder.block.13': 'cpu',
 'encoder.block.14': 'cpu',
 'encoder.block.15': 'cpu',
 'encoder.block.16': 'cpu',
 'encoder.block.17': 'cpu',
 'encoder.block.18': 'cpu',
 'encoder.block.19': 'cpu',
 'encoder.block.20': 'cpu',
 'encoder.block.

In [10]:
# pip install accelerate transformers bitsandbytes
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
model = T5ForConditionalGeneration.from_pretrained("google/flan-ul2", device_map=device_map, load_in_8bit=True, llm_int8_enable_fp32_cpu_offload=True, offload_folder="./offload")                                                                 
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

input_string = "Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?"                                               

inputs = tokenizer(input_string, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(inputs, max_length=200)

print(tokenizer.decode(outputs[0]))
# <pad> They have 23 - 20 = 3 apples left. They have 3 + 6 = 9 apples. Therefore, the answer is 9.</s>


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
Loading checkpoint shards:  12%|█▎        | 1/8 [00:04<00:30,  4.32s/it]

: 

: 